# Spectral Clustering  

Spectral clustering uses the eigenvectors of a Laplacian of the input graph to find a 
given number of clusters which satisfy a given quality metric.
Modularity Maximization is such a metric. 

@See:  https://en.wikipedia.org/wiki/Spectral_clustering

To perform spectral clustering using the modularity maximization metric in cugraph use:
__cugraph.spectralModularityMaximizationClustering(G, num_clusters, num_eigen_vects)__

Input
* G: A cugraph.Graph object
* num_clusters: The number of clusters to find
* num_eig: (optional) The number of eigenvectors to use

Returns
* __df__: cudf.DataFrame with two names columns:
    * df["vertex"]: The vertex id.
    * df["cluster"]: The assigned partition.
        

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)

Zachary used a min-cut flow model to partition the graph into two clusters, shown by the circles and squares.  Zarchary wanted just two cluster based on a conflict that caused the Karate club to break into two separate clubs.  Many social network clustering methods identify more that two social groups in the data.

In [1]:
# Import needed libraries
import cugraph
import cudf
import numpy as np
from scipy.io import mmread
from collections import OrderedDict

In [2]:
# Test file  - using the clasic Karate club dataset.  
datafile='../data/networks/karate-data.csv'

In [3]:
# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

In [4]:
# Need to shift the vertex IDs to start with zero rather than one (next version of cuGraph will fix this issue)
gdf["src_0"] = gdf["src"] - 1
gdf["dst_0"] = gdf["dst"] - 1
gdf["data"]  = 0

In [5]:
gdf.head().to_pandas()

,src,dst,src_0,dst_0,data
0,1,2,0,1,0
1,1,3,0,2,0
2,1,4,0,3,0
3,1,5,0,4,0
4,1,6,0,5,0


In [ ]:
#data = cudf.Series(np.zeros(len(gdf), dtype=np.float32))

In [6]:
# create a Graph 
Gc = cugraph.Graph()
Gc.add_edge_list(gdf["src_0"], gdf["dst_0"], gdf["data"])

In [7]:
# Call spectralModularityMaximizationClustering on the graph for 3 clusters
# using 3 eigenvectors:
df = cugraph.spectralModularityMaximizationClustering(Gc, 3, num_eigen_vects=3)

GDFError: b'GDF_UNSUPPORTED_DTYPE'

In [ ]:
# Read in the data file into scipy matrix format
mmFile='../data/networks/karate.mtx'
M = mmread(mmFile).asfptype().tolil()
M = M.tocsr()

In [ ]:
# Load the structure of the graph into GPU memory and create a CuGraph
# graph object:
row_offsets = cudf.Series(M.indptr)
col_indices = cudf.Series(M.indices)
values = cudf.Series(M.data)
G = cugraph.Graph()
G.add_adj_list(row_offsets, col_indices, values)

In [ ]:
# Call spectralModularityMaximizationClustering on the graph for 3 clusters
# using 3 eigenvectors:
df = cugraph.spectralModularityMaximizationClustering(G, 3, num_eigen_vects=3)

In [ ]:
# Check the modularity score for the produced clustering
score = cugraph.analyzeClustering_modularity(G, 2, df['cluster'])
score

In [ ]:
def print_cluster(df, id):
    
    # See which nodes are in cluster 0:
    part = []
    for i in range(len(df)):
        if (df['cluster'][i] == id):
            part.append(df['vertex'][i])
    print(part)

In [ ]:
# See which nodes are in cluster 0:
print_cluster(df, 0)

In [ ]:
print_cluster(df, 1)

In [ ]:
print_cluster(df, 2)

Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___